In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
import seaborn as sns

In [3]:
# Load the data
nordpool = pd.read_csv('New Data/2022/Nordpool DKK.csv')
energinet = pd.read_csv('New Data/2022/Energinet DKK.csv')
weather_observation = pd.read_csv('New Data/2022/Weather Observation.csv')
weather_forecast = pd.read_csv('New Data/2022/Weather Forecast.csv')
network = pd.read_csv('New Data/2022/Network Manager.csv')

# Filter energinet price area
energinet = energinet[energinet['PriceArea | PriceArea | 804696'] == 'DK2']
# Drop price area
energinet.drop('PriceArea | PriceArea | 804696', axis=1, inplace=True)

# Convert ts to datetime
nordpool['ts'] = pd.to_datetime(nordpool['ts'])
energinet['ts'] = pd.to_datetime(energinet['ts'])
weather_observation['ts'] = pd.to_datetime(weather_observation['ts'])
weather_forecast['ts'] = pd.to_datetime(weather_forecast['ts'])
network['ts'] = pd.to_datetime(network['ts'])

# Resample network_manager data to hourly intervals (average the data)
network = network.set_index('ts')
network = network.resample('H').mean()
network.reset_index(inplace=True)

/var/folders/tz/wvdldxqj00gc2cslzkpx77sr0000gn/T/ipykernel_98402/4139805231.py:22: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  network = network.resample('H').mean()


#### Data Handling

In [4]:
# Compare length of dfs + get first & last timestamp
def get_first_last(**dfs):
    # Initialize
    first_last = pd.DataFrame(index=['first', 'last', 'length', 'start_check', 'end_check'])
    col_name='ts'
    # Save first and last timestamp for each dataframe
    for name, df in dfs.items():
        first_ts = df['ts'].iloc[0]
        last_ts = df['ts'].iloc[-1]
        start_check = start == first_ts
        end_check = end == last_ts
        first_last[name] = [first_ts, last_ts, len(df), start_check, end_check]
    return first_last

# Initialize
start = pd.to_datetime('2022-01-01 00:00:00')
end = pd.to_datetime('2022-12-31 23:00:00')

print("Length of a year in hours:", 365*24)
get_first_last(nordpool=nordpool, energine=energinet, weather_observation=weather_observation, weather_forecast=weather_forecast, network=network)

Length of a year in hours: 8760


,nordpool,energine,weather_observation,weather_forecast,network
first,2021-12-31 00:00:00,2022-01-01 00:00:00,2022-01-01 00:00:00,2021-12-29 00:00:00,2022-01-01 00:00:00
last,2022-12-31 23:00:00,2022-12-31 23:00:00,2022-12-31 23:00:00,2022-12-31 23:00:00,2022-12-31 23:00:00
length,8711,8759,8539,8706,8760
start_check,False,True,True,False,True
end_check,True,True,True,True,True


In [229]:
nordpool.head(30) # -> missing 01/01/2022

,ts,Nordpool Elspot Prices - hourly price DK-DK2 DKK/MWh | 9F7J/00/00/Nordpool/DK2/hourly_spot_dkk | 4820
0,2021-12-31 00:00:00,"223,38"
1,2021-12-31 01:00:00,"194,46"
2,2021-12-31 02:00:00,"186,80"
3,2021-12-31 03:00:00,"178,54"
4,2021-12-31 04:00:00,"184,49"
5,2021-12-31 05:00:00,"181,67"
6,2021-12-31 06:00:00,"202,12"
7,2021-12-31 07:00:00,"225,61"
8,2021-12-31 08:00:00,"242,72"
9,2021-12-31 09:00:00,"261,90"


In [5]:
# Exclude all before 01/01/2022 from Nordpool
nordpool = nordpool[nordpool['ts'] >= start]
nordpool.reset_index(drop=True, inplace=True)
nordpool.head()

,ts,Nordpool Elspot Prices - hourly price DK-DK2 DKK/MWh | 9F7J/00/00/Nordpool/DK2/hourly_spot_dkk | 4820
0,2022-01-02 00:00:00,"258,27"
1,2022-01-02 01:00:00,"255,74"
2,2022-01-02 02:00:00,"255,52"
3,2022-01-02 03:00:00,"253,58"
4,2022-01-02 04:00:00,"219,82"


In [204]:
weather_forecast.head(30)  # jump from 29/12/2021 to 02/01/2022 -> missing 01/01/2022 + 02/01/2022 until 08:00

,ts,Weather forecast for the max air temperature for the coming 6 hours at Bornholm | 9F7P/00/00/MET-Norway/forecast/air_temperature_max | 128206,Weather forecast for wind direction at Bornholm | 9F7P/00/00/MET-Norway/forecast/wind_direction | 128270,Weather forecast for the precipitation amount for the coming 1 hour at Bornholm | 9F7P/00/00/MET-Norway/forecast/precipitation_amount_acc | 128238,Weather forecast for the relativehumidity amount at Bornholm | 9F7P/00/00/MET-Norway/forecast/relative_humidity_2m | 128254,Weather forecast for wind speed at Bornholm | 9F7P/00/00/MET-Norway/forecast/wind_speed | 128286,Weather forecast for wind speed y_direction at Bornholm | 9F7P/00/00/MET-Norway/forecast/y_wind_10m | 128318,Weather forecast for solar shortwave flux at Bornholm | 9F7P/00/00/MET-Norway/forecast/integral_of_surface_downwelling_shortwave_flux_in_air_wrt_time | 128334,Weather forecast for the minimum air temperature for the coming 6 hours at Bornholm | 9F7P/00/00/MET-Norway/forecast/air_temperature_min | 128222,Weather forecast for wind speed x_direction at Bornholm | 9F7P/00/00/MET-Norway/forecast/x_wind_10m | 128302,Weather forecast for the air temperature at Bornholm | 9F7P/00/00/MET-Norway/forecast/air_temperature_2m | 128190
0,2021-12-29 00:00:00,273.51764,161.68120,1.496094,0.980413,4.776667,4.849883,4.401408e+06,273.42554,-1.894686,274.34357
1,2021-12-29 01:00:00,273.53998,157.67975,1.902344,0.985037,3.154158,4.141306,4.493824e+06,274.28613,-1.199751,274.29535
2,2021-12-29 02:00:00,273.62848,166.18024,1.780273,0.983648,4.277684,2.970285,4.401408e+06,273.53967,-0.368376,274.38904
3,2021-12-29 03:00:00,274.49185,177.39850,2.115234,0.990713,3.711077,2.506868,4.401408e+06,274.39203,-0.802530,274.49158
4,2021-12-29 04:00:00,273.75880,173.32155,2.312500,0.996801,2.145191,2.038771,4.493824e+06,273.71320,-0.340435,274.50543
5,2021-12-29 05:00:00,273.83307,177.10390,2.474609,1.000006,2.258067,2.149813,4.401408e+06,273.76290,-0.770323,273.83624
6,2021-12-29 06:00:00,274.54816,158.37532,2.560547,1.000006,1.956588,1.819742,4.401408e+06,274.50742,-0.718338,274.50732
7,2021-12-29 07:00:00,274.50890,174.17190,2.583984,1.000004,1.332731,1.305176,4.493824e+06,273.86572,-0.065403,274.45227
8,2021-12-29 08:00:00,274.44824,89.60977,3.115234,1.000006,0.465735,-0.001333,4.494592e+06,274.06534,-0.274176,274.06396
9,2021-12-29 09:00:00,274.06050,229.50792,2.666016,1.000005,0.162841,0.105337,4.504832e+06,273.97067,0.123575,273.97296


In [6]:
# Exclude all before 01/01/2022 from Weather Forecast
weather_forecast = weather_forecast[weather_forecast['ts'] >= start]
weather_forecast.reset_index(drop=True, inplace=True)
weather_forecast.head()

,ts,Weather forecast for the max air temperature for the coming 6 hours at Bornholm | 9F7P/00/00/MET-Norway/forecast/air_temperature_max | 128206,Weather forecast for wind direction at Bornholm | 9F7P/00/00/MET-Norway/forecast/wind_direction | 128270,Weather forecast for the precipitation amount for the coming 1 hour at Bornholm | 9F7P/00/00/MET-Norway/forecast/precipitation_amount_acc | 128238,Weather forecast for the relativehumidity amount at Bornholm | 9F7P/00/00/MET-Norway/forecast/relative_humidity_2m | 128254,Weather forecast for wind speed at Bornholm | 9F7P/00/00/MET-Norway/forecast/wind_speed | 128286,Weather forecast for wind speed y_direction at Bornholm | 9F7P/00/00/MET-Norway/forecast/y_wind_10m | 128318,Weather forecast for solar shortwave flux at Bornholm | 9F7P/00/00/MET-Norway/forecast/integral_of_surface_downwelling_shortwave_flux_in_air_wrt_time | 128334,Weather forecast for the minimum air temperature for the coming 6 hours at Bornholm | 9F7P/00/00/MET-Norway/forecast/air_temperature_min | 128222,Weather forecast for wind speed x_direction at Bornholm | 9F7P/00/00/MET-Norway/forecast/x_wind_10m | 128302,Weather forecast for the air temperature at Bornholm | 9F7P/00/00/MET-Norway/forecast/air_temperature_2m | 128190
0,2022-01-02 09:00:00,279.84314,235.57857,0.000000,1.000007,5.813893,3.280224,0.0,279.84314,4.800692,279.84314
1,2022-01-02 10:00:00,279.87286,235.08603,0.024658,1.000007,5.404374,3.088070,20640.0,279.68402,4.435625,279.68396
2,2022-01-02 11:00:00,279.88293,243.83610,0.057129,1.000007,5.612103,2.468229,45728.0,279.68176,5.040297,279.86313
3,2022-01-02 12:00:00,280.35693,250.11731,0.096191,1.000007,5.622662,2.301142,62336.0,280.35693,5.346596,280.10898
4,2022-01-02 13:00:00,280.50568,247.23703,0.108887,1.000007,4.478304,2.164732,37184.0,280.34027,4.493957,280.50726


In [7]:
get_first_last(nordpool=nordpool, energinet=energinet, weather_observation=weather_observation, weather_forecast=weather_forecast, network=network)

,nordpool,energinet,weather_observation,weather_forecast,network
first,2022-01-02 00:00:00,2022-01-01 00:00:00,2022-01-01 00:00:00,2022-01-02 09:00:00,2022-01-01 00:00:00
last,2022-12-31 23:00:00,2022-12-31 23:00:00,2022-12-31 23:00:00,2022-12-31 23:00:00,2022-12-31 23:00:00
length,8687,8759,8539,8694,8760
start_check,False,True,True,False,True
end_check,True,True,True,True,True


In [243]:
# Merge the dataframes on network with left join
data = pd.merge(network, weather_observation, on='ts', how='left')
data = pd.merge(data, weather_forecast, on='ts', how='left')

# Sort the data by the timestamp column 'ts' to ensure time order
data = data.sort_values(by='ts')

data.head()

,ts,Hasle Common 10kV Lines Voltage | has_fel_10kvskinnespend | 804133,Hasle Vind Active Power | has_vin_effekt | 804123,Hasle Vind Current | has_vin_belastning | 804117,Observed maximum temperature past hour at Hammer Odde Fyr - DMI station 06193 | 9F7P/7Q/XC/DMI/metObs/temp_max_past1h/06193 | 406592,Observed mean intensity of global radiation in the latest hour at Hammer Odde Fyr - DMI station 06193 | 9F7P/7Q/XC/DMI/metObs/radia_glob_past1h/06193 | 406672,Observed minimum temperature past hour at Hammer Odde Fyr - DMI station 06193 | 9F7P/7Q/XC/DMI/metObs/temp_min_past1h/06193 | 406608,Observed mean wind speed the past hour at Hammer Odde Fyr - DMI station 06193 | 9F7P/7Q/XC/DMI/metObs/wind_speed_past1h/06193 | 406640,Observed mean humidity past hour at Hammer Odde Fyr - DMI station 06193 | 9F7P/7Q/XC/DMI/metObs/humidity_past1h/06193 | 406576,Observed mean temperature past hour at Hammer Odde Fyr - DMI station 06193 | 9F7P/7Q/XC/DMI/metObs/temp_mean_past1h/06193 | 406560,...,Weather forecast for the max air temperature for the coming 6 hours at Bornholm | 9F7P/00/00/MET-Norway/forecast/air_temperature_max | 128206,Weather forecast for wind direction at Bornholm | 9F7P/00/00/MET-Norway/forecast/wind_direction | 128270,Weather forecast for the precipitation amount for the coming 1 hour at Bornholm | 9F7P/00/00/MET-Norway/forecast/precipitation_amount_acc | 128238,Weather forecast for the relativehumidity amount at Bornholm | 9F7P/00/00/MET-Norway/forecast/relative_humidity_2m | 128254,Weather forecast for wind speed at Bornholm | 9F7P/00/00/MET-Norway/forecast/wind_speed | 128286,Weather forecast for wind speed y_direction at Bornholm | 9F7P/00/00/MET-Norway/forecast/y_wind_10m | 128318,Weather forecast for solar shortwave flux at Bornholm | 9F7P/00/00/MET-Norway/forecast/integral_of_surface_downwelling_shortwave_flux_in_air_wrt_time | 128334,Weather forecast for the minimum air temperature for the coming 6 hours at Bornholm | 9F7P/00/00/MET-Norway/forecast/air_temperature_min | 128222,Weather forecast for wind speed x_direction at Bornholm | 9F7P/00/00/MET-Norway/forecast/x_wind_10m | 128302,Weather forecast for the air temperature at Bornholm | 9F7P/00/00/MET-Norway/forecast/air_temperature_2m | 128190
0,2022-01-01 00:00:00,10.399732,4.402137,251.091703,5.7,1.0,5.5,8.2,100.0,5.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-01-01 01:00:00,10.414370,3.251096,185.044956,5.7,1.0,5.4,9.3,100.0,5.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-01-01 02:00:00,10.423121,3.608153,206.984440,5.9,1.0,5.1,9.3,100.0,5.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-01-01 03:00:00,10.421467,1.952956,112.202598,5.6,1.0,5.3,8.5,100.0,5.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2022-01-01 04:00:00,10.417432,0.813101,47.719737,5.5,1.0,5.2,7.1,100.0,5.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [267]:
data.isnull().sum()

ts                                                                           0
Hasle Common 10kV Lines Voltage                                            950
Hasle Vind Active Power                                                    947
Hasle Vind Current                                                         950
Observed maximum temperature past hour                                     318
Observed mean intensity of global radiation in the latest hour             386
Observed minimum temperature past hour                                     380
Observed mean wind speed the past hour                                     381
Observed mean humidity past hour                                           316
Observed mean temperature past hour                                        374
Observed mean wind direction the past hour                                 320
Observed accumulated precipitation the past hour                           321
Weather forecast for the max air temperature for the

In [252]:
# Column cleaning

# Use regex to remove everything starting from '|'
data.columns = data.columns.str.replace(r'\|.*', '', regex=True)
nordpool.columns = nordpool.columns.str.replace(r'\|.*', '', regex=True)
energinet.columns = energinet.columns.str.replace(r'\|.*', '', regex=True)

# Use regex to remove everything starting from the word "at"
data.columns = data.columns.str.replace(r'\sat.*', '', regex=True)

# Strip columns
data.columns = data.columns.str.strip()

data.columns

Index(['ts', 'Hasle Common 10kV Lines Voltage', 'Hasle Vind Active Power',
       'Hasle Vind Current', 'Observed maximum temperature past hour',
       'Observed mean intensity of global radiation in the latest hour',
       'Observed minimum temperature past hour',
       'Observed mean wind speed the past hour',
       'Observed mean humidity past hour',
       'Observed mean temperature past hour',
       'Observed mean wind direction the past hour',
       'Observed accumulated precipitation the past hour',
       'Weather forecast for the max air temperature for the coming 6 hours',
       'Weather forecast for wind direction',
       'Weather forecast for the precipitation amount for the coming 1 hour',
       'Weather forecast for the relativehumidity amount',
       'Weather forecast for wind speed',
       'Weather forecast for wind speed y_direction',
       'Weather forecast for solar shortwave flux',
       'Weather forecast for the minimum air temperature for the coming 6